# **Character-level Recurrent Neural Network**

In this mini-project we build a Character-level RNN for text generation trained on A. Pushkin Eugene Onegin.<br>

<img src="https://byronsmuse.wordpress.com/wp-content/uploads/2017/04/lidia-timoshenko-1903-1976-tatyana-and-onegin-years-later.jpg" width=40% style="border-radius:20px">

In [1]:
import torch
from torch import nn
from torch import optim
import numpy as np
from torch.nn import functional as F

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### **step 1. load the data**

In [3]:
data = open("onegin.txt", "r").read()
chars = sorted(set(data))
n_chars = len(chars)

itos = {i:s for i, s in enumerate(chars)}
stoi = {s:i for i, s in itos.items()}

n_chars

145

### **step 2. rnn**
Okay, how does RNN work?<br>
It takes one input token and one hidden_state token at a time.<br>
It concatenates them and passes through a Linear Layer with tanh activation.<br>
Finally, it is passed through final to_output transformation and returned.<br>
We optimize RNN to return expected characters.<br>
For character level RNN with data "hello" input-target mapping is the following:<br>
h -> e<br>
e -> l<br>
l -> l<br>
l -> o<br>

In [4]:
"""As you can see it's a tiny and simple code"""

class ShallowRNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    self.hidden_size = hidden_size
    self.i2h = nn.Linear(input_size + hidden_size, hidden_size)  # We concatenate input and hidden
    self.relu = nn.ReLU()
    self.h2o = nn.Linear(hidden_size, output_size)  # Final transformation

  def init_hidden(self, device):
    return torch.zeros((1, self.hidden_size), device=device)

  def forward(self, input, hidden):
    # shapes are (1, input) and (1, hidden) - one hot encoded
    inp_w_hidden = torch.cat([input, hidden], dim=1)  #We concatenate them along dim 1
    new_hidden = self.relu(self.i2h(inp_w_hidden))
    return self.h2o(new_hidden), new_hidden

Okay, we have our RNN, but how do we train it?<br>
As described above we we are working with one token at a time.<br>
This is to say, for epoch in range epochs we extract a random chunk of text of fixed length, encode it using one_hot_encoding, so the shape is (1, n_chars).<br>
For this block we create a dataset of format input, target with encoded chars and pass char after char sequentially.<br>
As you can see we have a ```init_hidden()``` method in RNN for a reason. On first step we initialize it with zeros, which is a right thing to do in our case.

In [5]:
from tqdm import tqdm
import random

In [28]:
# Generation Code used on trained model (and during training with logs=True)
def generate(model, input, len_, device):
  model.eval()
  with torch.inference_mode():
    chars = [stoi[input]]
    h = model.init_hidden(device)
    for i in range(len_):
      input = F.one_hot(torch.tensor([chars[-1]]), n_chars).to(device)
      logits, h = model(input, h)
      probs = torch.softmax(logits, dim=1)
      ix = torch.multinomial(probs, 1).item()
      chars.append(ix)
    return "".join([itos[ch] for ch in chars])

In [31]:
# I decided to make code more modular
# To experiment more with hyperparameters
def train_rnn(data, input_size, hidden_size, output_size, chunk_length=50,
              epochs=5_000, lr=0.001, logs=True, optimizer=None, device=device):

  # Variables set up
  rnn = ShallowRNN(input_size, hidden_size, output_size).to(device)
  if not optimizer:
    optimizer = optim.Adam(rnn.parameters(), lr)
  else:
    optimizer = optimizer(rnn.parameters(), lr)
  loss_fn = nn.CrossEntropyLoss()

  for epoch in range(epochs):
    rnn.train()
    # Data Extraction + preparation
    chunk_start_i = torch.randint(0, len(data) - chunk_length, (1,))
    chunk = data[chunk_start_i:(chunk_start_i + chunk_length)]
    encoded_chunk = torch.tensor([stoi[ch] for ch in chunk])
    ohe_chunk = F.one_hot(encoded_chunk, num_classes=n_chars).to(device)
    input = ohe_chunk[:-1].float()
    target = ohe_chunk[1:].float()
    hidden = rnn.init_hidden(device)
    chunk_loss = 0.0
    # Training
    for input_token, target_token in zip(input, target):
      logit, hidden = rnn(input_token.unsqueeze(0), hidden)
      token_loss = loss_fn(logit, target_token.unsqueeze(0))
      chunk_loss += token_loss
    # Optimization
    optimizer.zero_grad()
    chunk_loss.backward()
    optimizer.step()

    if epoch % 500 == 0 and logs:
      print(f"Epoch: {epoch} | Chunk Loss: {chunk_loss.item() / chunk_length}")
      print("Generated text:")
      print(generate(rnn, random.choice(chars), chunk_length, device))
      print("=" * 60)
  return rnn

In [32]:
rnn_model = train_rnn(data, n_chars, 512, n_chars)

Epoch: 0 | Chunk Loss: 4.881054992675781
Generated text:
oxдO2àаЕrожАk à"бwжaэДГЭТzъ«лê6в
айЖ"е’SezjгOкзcцeБ
Epoch: 500 | Chunk Loss: 2.9199087524414065
Generated text:
ыд шытом даано,
се дия а орых нролеиялятего,
Нал се
Epoch: 1000 | Chunk Loss: 2.563825988769531
Generated text:
Gяь.ей . ф шаи планьгалиносгив енов ери почан,
И . 
Epoch: 1500 | Chunk Loss: 2.2844252014160156
Generated text:
Ew больсянкай мнат
Пребась вже берчань нетдой днай 
Epoch: 2000 | Chunk Loss: 2.706788330078125
Generated text:
80I1)

XXXVI

За ни —те сь. Е жет вел: кол чере, . 
Epoch: 2500 | Chunk Loss: 2.3051292419433596
Generated text:
3B2тыйный, везмине
И кинит орялуш ойни тыят сипоньн
Epoch: 3000 | Chunk Loss: 2.3529917907714846
Generated text:
я.
Но о нео Тат омерпот манеросный
Уо похомне) мати
Epoch: 3500 | Chunk Loss: 2.2949143981933595
Generated text:
è во угдушой тазровала ,
Не рошиц, я ко Ьинья
Нам и
Epoch: 4000 | Chunk Loss: 1.2133419799804688
Generated text:
8фразыт поладутель слудилсы заболья бодсси

In [33]:
print(generate(rnn_model, "О", 500, device))

Он с «воерого сожда
Изгер когда какой деворое в В лут солно блуза приять, склуя я но сожицах. Ещи за с не е сом согласимое
спед понного коловам,
Не олегос,»
20XXXVII

Обнять вом они я сляский волько пастеннема вис не щеет, шлянны

Не хольеревы х Как фрац
Приск чуг.;
Заводу и протенля,
осто скре-мох коняте:
Ную! на но ты,
Она потях» и скуть
Как вспому твой надоди гоносеглясчик?
На жавна хручейной
За на тоспрама;
Просты;
Борь биетерь любуть рамнеку в толо на сужет? V... Мохосвя, потихой
Они глебете


**Our pipeline is not ideal, and the model is shallow, but our model learned the structure of the text!**

In [34]:
print(generate(rnn_model, "О", 1_000, device))

Отоин Онегин г и камой печны. Вылнум,
И топы смулнам и пнеретшью.
VII
 до-жила в рушин?
Дая Голи, тск начный
Котких претоморень
Имь соподо ори него оли заня;
В резечной попрывиц
В дехой;
ЛАй, дурозим смей;
Яснав иленило пуюн то приятить сву к маж като в поиль бездемий, раго, Оногин Фмла морвен...
XL.
сутальяныц Онегин замалдя доножит собьфая,
Проволь сечет башных тороком раккоклена там довочу тай Тамь пешех ничию3
Нимену мородише ж ны ницишет бозва тулою можел —
Пер:
Я но дварко,
Здесь ухого,
И грух мерук Твеши! плав,
Идень ских оне следа.
На ет соб нами; насв!
VIII

Евга, в соф Мелетали гонъе,
5таяе делий,
и стжу:  душу товыла подной пол, Фркдю завном, песто доны;
1еи Пусуга Венно сведный; и токровой, валь уланей,
Перикнайшит, подиса, издокно) боська, брудит
П
родьнсеночко, любял оне  мое-ныегин
К коятьяни обозй
юльгрова!
Нла песез,
Быль походу х соды не ждооки спак?
. . . . . ; Лобо мСе Лыбо в ни удоная.
Дотий
Стов зар мчестве?
Комей, в годаль он себлой в тупновь,
Не бля чня ж начны,

**It even mentions "Онегин" - main character**

### **Deep RNN**
Okay, shallow model performs well, but what if we train a deeper RNN with higher block length? Let's see!<br>
Deep RNN introduces multiple of hidden layers, meaning we need to keep track of all of them<br>

<img src="https://d2l.ai/_images/deep-rnn.svg">

In [53]:
"""I keep it simple, making all of hidden_sizes equal"""

class DeepRNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    self.hidden_size = hidden_size
    self.i_h1 = nn.Linear(input_size + hidden_size, hidden_size)
    self.relu = nn.ReLU()
    self.h1_h2 = nn.Linear(hidden_size * 2, hidden_size)
    self.relu = nn.ReLU()
    self.h2_h3 = nn.Linear(hidden_size * 2, hidden_size)
    self.relu = nn.ReLU()
    self.h3_o = nn.Linear(hidden_size, output_size)

  def init_hidden(self, device):
    return [torch.randn(1, self.hidden_size, device=device) for _ in range(3)]

  def forward(self, input, hiddens:tuple):
    h1, h2, h3 = hiddens

    input_w_h1 = torch.cat([input, h1], dim=1)
    new_h1 = self.relu(self.i_h1(input_w_h1))

    h1_h2 = torch.cat([new_h1, h2], dim=1)
    new_h2 = self.relu(self.h1_h2(h1_h2))

    h2_h3 = torch.cat([new_h2, h3], dim=1)
    new_h3 = self.relu(self.h2_h3(h2_h3))

    h3_o = self.h3_o(new_h3)

    return h3_o, (new_h1, new_h2, new_h3)

In [54]:
def generate(model, input, len_, device):
  model.eval()
  with torch.inference_mode():
    chars = [stoi[input]]
    (h1, h2, h3) = model.init_hidden(device)
    for i in range(len_):
      input = F.one_hot(torch.tensor([chars[-1]]), n_chars).to(device)
      logits, (h1, h2, h3) = model(input, (h1, h2, h3))
      probs = torch.softmax(logits, dim=1)
      ix = torch.multinomial(probs, 1).item()
      chars.append(ix)
    return "".join([itos[ch] for ch in chars])

In [59]:
common_chars = list("абвгдежзиклмнопрстуфхцчшщэюя")

In [60]:
# Sadly I didn't foresee it...
def train_rnn(data, input_size, hidden_size, output_size, chunk_length=50,
              epochs=5_000, lr=0.001, logs=True, optimizer=None, device=device):

  # Variables set up
  rnn = DeepRNN(input_size, hidden_size, output_size).to(device)
  if not optimizer:
    optimizer = optim.Adam(rnn.parameters(), lr)
  else:
    optimizer = optimizer(rnn.parameters(), lr)
  loss_fn = nn.CrossEntropyLoss()

  for epoch in range(epochs):
    rnn.train()
    # Data Extraction + preparation
    chunk_start_i = torch.randint(0, len(data) - chunk_length, (1,))
    chunk = data[chunk_start_i:(chunk_start_i + chunk_length)]
    encoded_chunk = torch.tensor([stoi[ch] for ch in chunk])
    ohe_chunk = F.one_hot(encoded_chunk, num_classes=n_chars).to(device)
    input = ohe_chunk[:-1].float()
    target = ohe_chunk[1:].float()
    h1, h2, h3 = rnn.init_hidden(device)
    chunk_loss = 0.0
    # Training
    for input_token, target_token in zip(input, target):
      logit, (h1, h2, h3) = rnn(input_token.unsqueeze(0), (h1, h2, h3))
      token_loss = loss_fn(logit, target_token.unsqueeze(0))
      chunk_loss += token_loss
    # Optimization
    optimizer.zero_grad()
    chunk_loss.backward()
    optimizer.step()

    if epoch % 500 == 0 and logs:
      print(f"Epoch: {epoch} | Chunk Loss: {chunk_loss.item() / chunk_length}")
      print("Generated text:")
      print(generate(rnn, random.choice(common_chars), chunk_length, device))  # I decided to use good chars
      print("=" * 60)
  return rnn

In [61]:
rnn_model = train_rnn(data, n_chars, 512, n_chars, chunk_length=150)

Epoch: 0 | Chunk Loss: 4.952339680989583
Generated text:
иФ
Ll ;FэШnniиècДЦ9:Х)uпpГги5ьâнp.вçW1rY9*v7fL".Hêь,Л1aGvp
èйпБЦakъdсMd1JЗqЛ
з5"пYз4pсQhОС3lqàâEXТj-FЗmEf;iГъ5F1у8Рd7NуDГн;pш*!п98фlëХzjдоyvЗч ЧXкaрêët
Epoch: 500 | Chunk Loss: 2.5578159586588542
Generated text:
ши горой
Онас, ставами дору: и рашерь, нопнывнукино.
Кыма моят дартае дло кеша чиля—
III

На качь!.
— сностя срония итит сошел?»
XVI

Покса евнуе не те
Epoch: 1000 | Chunk Loss: 2.7156876627604167
Generated text:
ньнохаленьийу,
Имкан, Лький!
XLIIII

Обвыго баяту зинное,
Вдорох,
И глешом6
В освну голпы — обдорил хротрыхны кумуцкой, покоти приуренлунье,
И кажно пр
Epoch: 1500 | Chunk Loss: 2.747464599609375
Generated text:
мм естаят.
Онустя грягой неводель,
Сяцы увигда чесивь.
Зарый плодес су селцы,
Но ни свелим мелицясь мосснятмянрень анорит отолорый-сальстем дродит. На 
Epoch: 2000 | Chunk Loss: 2.39908447265625
Generated text:
эь регда скобли нас под дде струстра.
На ни муд;
Малавиний отродан.
На пылти.) Онегиний Носей м

In [64]:
print(generate(rnn_model, "О", 1_000, device))

ОнеТет за пошлую
С нейта порапов.
Поглудает, я головой пит,
Они вылдувские брат,
Провяла сыв совела
И Логтые Ненькой с пуле
Усня подомам толке. Онегин клон быяшны,
Татьяна встречая Колил; тревижу;
Не с толки Онегин молво:
И писахеенье сколя,
И там и говеря...»
На в тапиты соседа емут на.
Я смоашал молодой
И деревне хроманям и лимавай.
V

Но подскигу  и станали,
Для нароны не подошалый завед.
XXXIV

И чувственно детя света слышат быть в ваши зонос
И взолагия ремы!
На виром садемать душал, ктося,
В меня суссену, душал и рочно как накодет
И челсквый мягчивый слову:
(Нугой люной,
И назтрадает, не нипяла,
С перед судненьей и с ниф взор,
Яли могуескою льют, покаресты в предногов
Пагам под ней,
Вашее ночкой рози,
Едва в мало, безмосла новый,
Про кативною приина с сенною
Там дуручела
И борот, и низный! Одесеньем
И там и венише и приилбонят
И в тими по не блесташесто бьи много метредальсь обовит.
Она проминшивый черть.
XXV

«Кто Лену на тет злыманий душой Клибытур,
Начему, искался мой не:
Еще ч

**Deep RNN didn't improve the situation drastically, but it didn't do worse and learned more words.**
<br>Seriously, model learned a lot of words and even constructions!<br>
This is pretty pog, especially we achieved this result so quickly and easily!<br>
Hopefully, this was helpful to u.<br>
In the next notebook we will compare **RNN**, **LSTM** and **GRU** to find the best performing model, then we will use word embeddings and train the best model possible!<br>
But this is it here!<br>
Oh, and let's have a tradition of coolness rate in every miniproject!<br>
**Coolness rate: 90%**
<br>
<img src="https://static.wikia.nocookie.net/cyberpunk/images/d/de/Johnny_Silverhand_Database_CP2077.png/revision/latest?cb=20231003222545" width=13%>